<a href="https://colab.research.google.com/github/sourcecode369/Kaggle-Kernels/blob/master/Twitter%20Sentiment%20Extraction/Twitter_Sentiment_Extraction_Text_Summarization_Approcah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files; files.upload()

!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle 
!chmod 600 ~/.kaggle/kaggle.json 

!kaggle competitions download -c tweet-sentiment-extraction

Saving kaggle.json to kaggle.json
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 86.1MB/s]
  0% 0.00/308k [00:00<?, ?B/s]
100% 308k/308k [00:00<00:00, 102MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 44.0MB/s]


In [0]:
!unzip *.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
!rm -r *.zip

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import unicodedata
import re
import numpy as np
import pandas as pd
import io
import os
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

train = train.dropna()

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  #w = unicode_to_ascii(w.lower().strip())
  # w = re.sub(r"([?.!,¿])", r" \1 ", w)
  # w = re.sub(r'[" "]+', " ", w)
  # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  #w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def create_dataset(data):
  
  train_text = train.text.apply(lambda x: preprocess_sentence(x))

  summary = train.selected_text.apply(lambda x: preprocess_sentence(x))

  test_text = test.text.apply(lambda x: preprocess_sentence(x))

  return text, summary, test_text

In [0]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters=''
  )
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
  return tensor, lang_tokenizer

In [0]:
def load_dataset(data, num_examples=None):
  text, summary, test = create_dataset(data)
  new_text = pd.concat([text,test],axis=0)
  new_text_tensor, text_tokenizer = tokenize(new_text)
  text_tensor, test_tensor = new_text_tensor[:len(text)],new_text_tensor[len(text):]
  summary_tensor, summary_tokenizer = tokenize(summary)
  return text_tensor, text_tokenizer, summary_tensor, summary_tokenizer, test_tensor

In [0]:
text_tensor, text_tokenizer, summary_tensor, summary_tokenizer, test_tensor = load_dataset(train)
max_length_text, max_length_summary = max_length(text_tensor), max_length(summary_tensor)

In [0]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(text_tensor, summary_tensor, test_size=0.1)

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

print ("Input Language; index to word mapping")
convert(text_tokenizer, input_tensor_train[1])
print ()
print ("Target Language; index to word mapping")
convert(summary_tokenizer, target_tensor_train[1])

Input Language; index to word mapping
2 ----> <start>
7 ----> the
24466 ----> eldorado
228 ----> house
8 ----> ,
1123 ----> goodbye
24467 ----> aliante
228 ----> house
1 ----> .
1509 ----> memories
219 ----> made
8 ----> ,
1509 ----> memories
11109 ----> fade
5 ----> !
3 ----> <end>

Target Language; index to word mapping
1 ----> <start>
256 ----> @
5 ----> the
27275 ----> eldorado
4664 ----> house,
1068 ----> goodbye
27276 ----> aliante
1546 ----> house.
27277 ----> *memories
27278 ----> made,
1956 ----> memories
27279 ----> fade!
2 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(text_tokenizer.word_index)+1
vocab_tar_size = len(summary_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 107]), TensorShape([64, 35]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 107, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(50)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 107, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 28792)


In [0]:
optimizer = tf.keras.optimizers.RMSprop()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([summary_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 15

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.3470
Epoch 1 Batch 100 Loss 1.6912
Epoch 1 Batch 200 Loss 1.3856
Epoch 1 Batch 300 Loss 1.6120
Epoch 1 Loss 1.6109
Time taken for 1 epoch 228.61747670173645 sec

Epoch 2 Batch 0 Loss 1.6639
Epoch 2 Batch 100 Loss 1.6901
Epoch 2 Batch 200 Loss 1.4969
Epoch 2 Batch 300 Loss 1.4805
Epoch 2 Loss 1.4478
Time taken for 1 epoch 192.06680011749268 sec

Epoch 3 Batch 0 Loss 1.1572
Epoch 3 Batch 100 Loss 1.2419
Epoch 3 Batch 200 Loss 1.0977
Epoch 3 Batch 300 Loss 1.1533
Epoch 3 Loss 1.1917
Time taken for 1 epoch 190.8835747241974 sec

Epoch 4 Batch 0 Loss 1.0526
Epoch 4 Batch 100 Loss 0.8540
Epoch 4 Batch 200 Loss 0.9529
Epoch 4 Batch 300 Loss 1.0766
Epoch 4 Loss 1.0049
Time taken for 1 epoch 192.1989049911499 sec

Epoch 5 Batch 0 Loss 0.9069
Epoch 5 Batch 100 Loss 0.7647
Epoch 5 Batch 200 Loss 0.8370
Epoch 5 Batch 300 Loss 0.9334
Epoch 5 Loss 0.9093
Time taken for 1 epoch 190.53616857528687 sec

Epoch 6 Batch 0 Loss 0.7260
Epoch 6 Batch 100 Loss 1.0119
Epoch 6 Batch 200 L

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_summary, max_length_text))

  sentence = preprocess_sentence(sentence)
  print(sentence)
  inputs = [text_tokenizer.word_index[i] for i in sentence.lower().split()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_text,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([summary_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_summary):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += summary_tokenizer.index_word[predicted_id] + ' '

    if summary_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
import matplotlib.ticker as ticker
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def summarize(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  count = 0
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  if count < 10:
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))
  yield count

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
test_summaries = test.text.apply(lambda x: summarize(x))
sample['selected_text'] 